###  It is required to import UMLS model into Neo4J before proceeding with the code

Refer to the Github code to import UMLS data into Neo4J :  

https://github.com/MIS-GrApH-AI/Public/blob/main/Data_import/UMLS_2020AB.ipynb

# Importing MED-RT into UMLS model

To download the MED-RT data visit the link -  https://evs.nci.nih.gov/ftp1/MED-RT/

Download the XML.zip files. 

MED-RT documentation can be read from here - https://evs.nci.nih.gov/ftp1/MED-RT/MED-RT%20Documentation.pdf

###  Refer to the procedure to get Association.csv

Use the following codes in your Neo4J browser : (You must change the file retrieve location according to where you have stored the MED-RT XML file)




Code 1:

WITH "CALL apoc.load.xml(\"file:///C:\\Users\\pc\\Desktop\\C-BIA\\Healthcare\\MED-RT\\Database\\Core_MEDRT_XML\\Parts\\Core_MEDRT_2021.07.06_XML.xml\" , '/terminology/association') YIELD value as association WITH [attr IN association._children WHERE attr._type IN ['namespace','name','from_namespace','from_name','from_code','to_namespace','to_name','to_code'] | attr._text] as pairs, [item in association._children WHERE item._type = \"qualifier\"] AS q1 RETURN pairs[0] as namespace, pairs[1] as name, pairs[2] as from_namespace, pairs[3] as from_name, pairs[4] as from_code, pairs[5] as to_namespace, pairs[6] as to_name, pairs[7] as to_code, q1[0]._children[0]._text as qualifier0_namespace, q1[0]._children[1]._text as qualifier0_name, q1[0]._children[2]._text as qualifier0_value, q1[1]._children[0]._text as qualifier1_namespace, q1[1]._children[1]._text as qualifier1_name, q1[1]._children[2]._text as qualifier1_value; " as query
CALL apoc.export.csv.query(query, "association.csv", {})
YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data;

Once this action is performed, "association.csv" will be created in your Neo4J dbms folder.

It is recommended to move this file to another easily accessible location since we will be referencing it in the upcoming code 

Similar actions must be performed for Code 2 given below

Code 2:

WITH "CALL apoc.load.xml(\"file:///C:\\Users\\pc\\Desktop\\C-BIA\\Healthcare\\MED-RT\\Database\\Core_MEDRT_XML\\Parts\\Core_MEDRT_2021.07.06_XML.xml\" , '/terminology/namespace') YIELD value as namespace WITH  [attr IN namespace._children WHERE attr._type IN ['name','code','version','authority'] | attr._text] as pairs RETURN  pairs[0] as name, pairs[1] as code, pairs[2] as version, pairs[3] as authority;" as query
CALL apoc.export.csv.query(query, "namespace.csv", {})
YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data;


The name of the csv file produced is "namespace.csv"

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
# Load MRCONSO.RRF into a dataframe
mrconso = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/UMLS/UMLS_Data/2021AA/META/MRCONSO.RRF', sep='|', header=None, encoding='utf-8')
mrconso[:5]

In [ ]:
mrconso.columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF', '']

In [ ]:
mrconso.drop(labels=['SUPPRESS', 'CODE', 'CVF','SAUI','SDUI', ''], axis=1, inplace=True)

In [ ]:
mrconso.info()

In [ ]:
mrconso[:5]

### Testing Merge - MRCONSO on Association


We perform merging of the association file to MRCONSO (UMLS) to match the CUI (UMLS) to the 'from_code' and the 'to_code' (SCUI in UMLS). This mapping is performed because our UMLS model nodes can be accessed with the CUI.
Once we get the CUI mapping we can create the relationships (the column 'name' in association) and add our namespace (MED-RT in this case) and its verison. 

The files "association.csv" and "namespace.csv" file locations must be referenced here 

In [ ]:
#Association file is used for merging MRCONSO (UMLS) with MED-RT data
association = pd.read_csv(r'C:\Users\pc\Desktop\C-BIA\Healthcare\MED-RT\XML - CSV\association.csv')

In [ ]:
#Version file is used for referencing the current MED-RT version into our UMLS model
version = pd.read_csv(r'C:\Users\pc\Desktop\C-BIA\Healthcare\MED-RT\XML - CSV\namespace.csv')

We create rectified namespace columns to change the namespace values to how it is in the UMLS data.

In [ ]:
association['from_namespace_rectified'] = association['from_namespace']
association['to_namespace_rectified'] = association['to_namespace']

In [ ]:
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('RxNorm','RXNORM')
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('MeSH','MSH')
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('SNOMED CT','SNOMEDCT_US')

In [ ]:
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('RxNorm','RXNORM')
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('MeSH','MSH')
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('SNOMED CT','SNOMEDCT_US')

In [ ]:
mrconsosub = mrconso[['SCUI','SAB','CUI']]

In [ ]:
mrconsosub.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
mrconsosub.to_csv('mrconsosub.csv', index=False, header=1, encoding='utf-8')

We merge the two dataframes based on the matching column data

In [ ]:
asso_merge = pd.merge(association, mrconsosub, left_on=['from_code','from_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [ ]:
asso_merge['from_CUI'] = asso_merge['CUI']

In [ ]:
asso_merge['from_SAB'] = asso_merge['SAB']

In [ ]:
asso_merge['from_SCUI'] = asso_merge['SCUI']

In [ ]:
asso_merge.drop(labels=['SCUI','SAB','CUI'], axis=1, inplace=True)

In [ ]:
asso_merge.drop_duplicates()

In [ ]:
asso_merge_final = pd.merge(asso_merge, mrconsosub, left_on=['to_code','to_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [ ]:
asso_merge_final['to_CUI'] = asso_merge_final['CUI'] 

In [ ]:
asso_merge_final['to_SAB'] = asso_merge_final['SAB']

In [ ]:
asso_merge_final['to_SCUI'] = asso_merge_final['SCUI']

In [ ]:
asso_merge_final.drop(labels=['SCUI','SAB','CUI','from_SAB','to_SAB','from_SCUI','to_SCUI'], axis=1, inplace=True)

In [ ]:
vers = version['version'].astype(str)
asso_merge_final['version'] = vers[0]
#asso_merge_final['version'] = asso_merge_final['version'].replace(asso_merge_final['version'],vers[0])

In [ ]:
asso_merge_final.to_csv('Association_Merged_MEDRT.csv', index=False, header=1, encoding='utf-8')

##### Move the "Association_Merged into the Neo4J import folder

### Creating relationships of MED-RT data into UMLS model

In [ ]:
import pandas as pd

In [ ]:
import getpass
password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")

In [ ]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j',password))
session=driver.session()

In [ ]:
# Create a node for each concept in the UMLS
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///CUIs_preferred_terms.csv" AS COLUMN CREATE (:Concept_UMLS {preferred_term:COLUMN[1], cui:COLUMN[0], semantic_type:COLUMN[2], UMLS_edition:'2021AA'})'''

#session.run(query)

In [ ]:
# Create a uniqueness constraint for the CUI property of each Concept_UMLS node
#query = 'CREATE CONSTRAINT UniqueCUIforConceptConstraint ON (c:Concept_UMLS) ASSERT c.cui IS UNIQUE'
#session.run(query)

In [ ]:
# Create the UMLS string nodes
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///str_to_CUI.csv" AS COLUMN CREATE (s:String_UMLS {string:COLUMN[0], cui:COLUMN[1], semantic_type:COLUMN[2]})'''

#session.run(query)

In [ ]:
command = '''USING PERIODIC COMMIT 100000 LOAD CSV WITH HEADERS FROM "file:///Association_Merged_MEDRT.csv" AS COLUMN MATCH (c1:Concept_UMLS {cui:COLUMN.from_CUI}) MATCH (c2:Concept_UMLS {cui:COLUMN.to_CUI}) OPTIONAL MATCH (c1)-[rel]->(c2) WITH c1, c2, COLUMN, COLLECT(TYPE(rel)) AS relTypes WHERE NOT COLUMN.name IN relTypes CALL apoc.create.relationship(c1, COLUMN.name, {source:COLUMN.namespace, version:COLUMN.version}, c2) YIELD rel as newrel RETURN c1, c2, newrel;'''
session.run(command)

### If you want to delete self loop relation - Synonym_Of 

###### Note - This can be used to remove any other self loop relationships as well, just replace 'Synonym Of' with the relationship name 

In [ ]:
#Delete self-loop relationship with name "Synonym_Of"

#command = '''MATCH p=(n1)-[r:`Synonym Of`]->(n2) WHERE n1.cui = n2.cui DELETE r'''
#session.run(command)

###### Once you reach this point , you can delete all the existing variables. It is preferable to do so if your system cannot handle executing all of these codes in one go

# Importing MeSH into UMLS model

To download the MeSH data visit the link -  https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/

For our code we are using desc2021.xml and supp2021.xml 

More information on MeSH can be found here - https://www.nlm.nih.gov/mesh/meshhome.html

### Refer to the procedure to get MeSH_Association_desc.csv  

Use the following codes in your Neo4J browser : (You must change the file retrieve location according to where you have stored the MeSH XML file)

Code 1:

WITH "CALL apoc.load.xml(\"file:///C:\\Users\\pc\\Desktop\\C-BIA\\Healthcare\\MeSH\\XML files\\Data\\desc2021.xml\" , '/DescriptorRecordSet/DescriptorRecord') YIELD value UNWIND value as desrec WITH [item in desrec._children WHERE item._type = \"DescriptorUI\"] AS Q, [item in desrec._children WHERE item._type = \"ConceptList\"] AS conceptlist1, desrec AS desrec UNWIND conceptlist1 AS conceptlist WITH [item in conceptlist._children WHERE item._type = \"Concept\"] AS concept1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist UNWIND concept1 AS concept WITH [item in concept._children WHERE item._type = \"ConceptUI\"] AS ConceptUI, [item in concept._children WHERE item._type = \"ConceptRelationList\"] AS CRL1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept UNWIND CRL1 AS CRL WITH [item in CRL._children WHERE item._type = \"ConceptRelation\"] AS CR1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept,  ConceptUI AS  ConceptUI, CRL AS CRL UNWIND CR1 AS CR WITH [item in CR._children WHERE item._type = \"Concept1UI\"] AS C1UI, [item in CR._children WHERE item._type = \"Concept2UI\"] AS C2UI, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept,  ConceptUI AS  ConceptUI, CRL AS CRL, CR AS CR RETURN DISTINCT \"MeSH\" AS namespace, CR.RelationName AS name, \"MeSH\" AS from_namespace, C1UI[0]._text AS from_code, \"MeSH\" AS to_namespace, C2UI[0]._text AS to_code, \"2021_2021.01.01\" AS version;" as query
CALL apoc.export.csv.query(query, "MeSH_association_desc.csv", {})
YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data;


Once this action is performed, "MeSH_association_desc.csv" will be created in your Neo4J dbms folder.

It is recommended to move this file to another easily accessible location since we will be referencing it in the upcoming code 

Code 2:

WITH "CALL apoc.load.xml(\"file:///C:\\Users\\pc\\Desktop\\C-BIA\\Healthcare\\MeSH\\XML files\\Data\\supp2021.xml\" , '/SupplementalRecordSet/ SupplementalRecord') YIELD value UNWIND value as desrec WITH [item in desrec._children WHERE item._type = \"SupplementalRecordUI\"] AS Q, [item in desrec._children WHERE item._type = \"ConceptList\"] AS conceptlist1, desrec AS desrec UNWIND conceptlist1 AS conceptlist WITH [item in conceptlist._children WHERE item._type = \"Concept\"] AS concept1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist UNWIND concept1 AS concept WITH [item in concept._children WHERE item._type = \"ConceptUI\"] AS ConceptUI, [item in concept._children WHERE item._type = \"ConceptRelationList\"] AS CRL1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept UNWIND CRL1 AS CRL WITH [item in CRL._children WHERE item._type = \"ConceptRelation\"] AS CR1, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept,  ConceptUI AS  ConceptUI, CRL AS CRL UNWIND CR1 AS CR WITH [item in CR._children WHERE item._type = \"Concept1UI\"] AS C1UI, [item in CR._children WHERE item._type = \"Concept2UI\"] AS C2UI, Q AS Q, desrec AS desrec, conceptlist AS conceptlist, concept AS concept,  ConceptUI AS  ConceptUI, CRL AS CRL, CR AS CR RETURN DISTINCT \"MeSH\" AS namespace, CR.RelationName AS name, \"MeSH\" AS from_namespace, C1UI[0]._text AS from_code, \"MeSH\" AS to_namespace, C2UI[0]._text AS to_code, \"2021_2021.01.01\" AS version;" as query
CALL apoc.export.csv.query(query, "MeSH_association_supp.csv", {})
YIELD file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data
RETURN file, source, format, nodes, relationships, properties, time, rows, batchSize, batches, done, data;


###### Code 2 requires a higher amount of heap space. It is suggested to perform the steps below used on MeSH_association_desc for MeSH_association_supp as well

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
# Load MRCONSO.RRF into a dataframe
mrconso = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/UMLS/UMLS_Data/2021AA/META/MRCONSO.RRF', sep='|', header=None, encoding='utf-8')
mrconso[:5]

In [ ]:
mrconso.columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF', '']

In [ ]:
mrconso.drop(labels=['SUPPRESS', 'CODE', 'CVF','SAUI','SDUI', ''], axis=1, inplace=True)

In [ ]:
mrconso.info()

In [ ]:
mrconso[:5]

#### Merge - MRCONSO on Association


We perform merging of the association file to MRCONSO (UMLS) to match the CUI (UMLS) to the 'from_code' and the 'to_code' (SCUI in UMLS). This mapping is performed because our UMLS model nodes can be accessed with the CUI.
Once we get the CUI mapping we can create the relationships (the column 'name' in association) and add our namespace (MED-RT in this case) and its verison.

In [ ]:
#This file contains columns that can be mapped with UMLS data
association = pd.read_csv(r'C:\Users\pc\Desktop\C-BIA\Healthcare\MeSH\XML - CSV\MeSH_association_desc.csv')

We create rectified namespace columns to change the namespace values to how it is in the UMLS data.

In [ ]:
association['from_namespace_rectified'] = association['from_namespace']
association['to_namespace_rectified'] = association['to_namespace']

In [ ]:
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('RxNorm','RXNORM')
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('MeSH','MSH')
association['from_namespace_rectified'] = association['from_namespace_rectified'].replace('SNOMED CT','SNOMEDCT_US')

In [ ]:
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('RxNorm','RXNORM')
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('MeSH','MSH')
association['to_namespace_rectified'] = association['to_namespace_rectified'].replace('SNOMED CT','SNOMEDCT_US')

In [ ]:
mrconsosub = mrconso[['SCUI','SAB','CUI']]

In [ ]:
mrconsosub.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
mrconsosub.to_csv('mrconsosub.csv', index=False, header=1, encoding='utf-8')

In [ ]:
asso_merge = pd.merge(association, mrconsosub, left_on=['from_code','from_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [ ]:
asso_merge['from_CUI'] = asso_merge['CUI']

In [ ]:
asso_merge['from_SAB'] = asso_merge['SAB']

In [ ]:
asso_merge['from_SCUI'] = asso_merge['SCUI']

In [ ]:
asso_merge.drop(labels=['SCUI','SAB','CUI'], axis=1, inplace=True)

In [ ]:
asso_merge.drop_duplicates()

In [ ]:
asso_merge_final = pd.merge(asso_merge, mrconsosub, left_on=['to_code','to_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [ ]:
asso_merge_final['to_CUI'] = asso_merge_final['CUI'] 

In [ ]:
asso_merge_final['to_SAB'] = asso_merge_final['SAB']

In [ ]:
asso_merge_final['to_SCUI'] = asso_merge_final['SCUI']

In [ ]:
asso_merge_final.drop(labels=['SCUI','SAB','CUI','from_SAB','to_SAB','from_SCUI','to_SCUI'], axis=1, inplace=True)

In [ ]:
#vers = version['version'].astype(str)
#asso_merge_final['version'] = vers[0]
#asso_merge_final['version'] = asso_merge_final['version'].replace(asso_merge_final['version'],vers[0])

In [ ]:
asso_merge_final.to_csv('MeSH_Association_Merged.csv', index=False, header=1, encoding='utf-8')

##### Move the "MeSH_Association_Merged" into the Neo4J import folder

### Creating relationships of MeSH data into UMLS model

In [ ]:
import pandas as pd

In [ ]:
import getpass
password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")

In [ ]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j',password))
session=driver.session()

In [ ]:
# Create a node for each concept in the UMLS
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///CUIs_preferred_terms.csv" AS COLUMN CREATE (:Concept_UMLS {preferred_term:COLUMN[1], cui:COLUMN[0], semantic_type:COLUMN[2], UMLS_edition:'2021AA'})'''

#session.run(query)

In [ ]:
# Create a uniqueness constraint for the CUI property of each Concept_UMLS node
#query = 'CREATE CONSTRAINT UniqueCUIforConceptConstraint ON (c:Concept_UMLS) ASSERT c.cui IS UNIQUE'
#session.run(query)

In [ ]:
# Create the UMLS string nodes
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///str_to_CUI.csv" AS COLUMN CREATE (s:String_UMLS {string:COLUMN[0], cui:COLUMN[1], semantic_type:COLUMN[2]})'''

#session.run(query)

###### Sometimes python causes issue and the command below is not executed. In case such a thing happens , it's recommended to run the command again.

To check if these specific relations were added, search in the Neo4J browser for relationship by the name of 'NRW' , 'BRD' , 'REL'. If these 3 exist , most probably all the relationships from the MeSH_Association_Merged.csv have been added.

Use the following code to check if the relationship exist: (you can replace REL with the name of the relationship)

MATCH p=()-[r:REL]->() RETURN p LIMIT 25

In [ ]:
command = '''USING PERIODIC COMMIT 100000 LOAD CSV WITH HEADERS FROM "file:///MeSH_Association_Merged.csv" AS COLUMN MATCH (c1:Concept_UMLS {cui:COLUMN.from_CUI}) MATCH (c2:Concept_UMLS {cui:COLUMN.to_CUI}) OPTIONAL MATCH (c1)-[rel]->(c2) WITH c1, c2, COLUMN, COLLECT(TYPE(rel)) AS relTypes WHERE NOT COLUMN.name IN relTypes CALL apoc.create.relationship(c1, COLUMN.name, {source:COLUMN.namespace, version:COLUMN.version}, c2) YIELD rel as newrel RETURN c1, c2, newrel;'''
session.run(command)

###### Once you reach this point , you can delete all the existing variables. It is preferable to do so if your system cannot handle executing all of these codes in one go

# Importing RxNorm into UMLS Model

To download the MeSH data you must have an account on the NLM.NIH official site 

RxNorm technical documentation - https://www.nlm.nih.gov/research/umls/rxnorm/docs/techdoc.html#s1_0

We will perform merging of two files - RXNREL.RRF and RXNCONSO.RRF.

This is necessary to get the necessary columns for further merging with MRCONSO (UMLS) so we can import our RxNorm data into our UMLS model

In [ ]:
# Load RXNREL.RRF into a dataframe
rxnrel = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/RxNorm/Data/rrf/RXNREL.RRF', sep='|', header=None, encoding='utf-8')
rxnrel[:5]

In [ ]:
rxnrel.columns = ['RXCUI1', 'RXAUI1', 'STYPE1', 'REL', 'RXCUI2', 'RXAUI2', 'STYPE2', 'RELA', 'RUI', 'SRUI', 'SAB', 'SL', 'RG☨', 'DIR☦', 'SUPPRESS', 'CVF', '']

In [ ]:
rxnrel.info()

In [ ]:
rxnrel[:5]

In [ ]:
# Load RXNCONSO.RRF into a dataframe
rxnconso = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/RxNorm/Data/rrf/RXNCONSO.RRF', sep='|', header=None, encoding='utf-8')
rxnconso[:5]

In [ ]:
rxnconso.columns = ['RXCUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'RXAUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF', '']

In [ ]:
rxnconso[:5]

In [ ]:
rxnconso.info()

In [ ]:
rxnrel.info()

In [ ]:
check = ['788773']
rxnconso[rxnconso.RXAUI.isin(check)]

In [ ]:
rxnrelsub = rxnrel
rxnconsosub = rxnconso

In [ ]:
rxnconsosub.drop(labels=['TS', 'STT', 'ISPREF', 'SRL', 'LUI', 'SUI', 'SUPPRESS', 'CVF','SAUI','SDUI', ''], axis=1, inplace=True)

In [ ]:
rxnrelsub.drop(labels=['DIR☦', 'RG☨', 'SRUI', 'SL', 'SUPPRESS', 'CVF', ''], axis=1, inplace=True)

In [ ]:
rxnrelsub[:5]

In [ ]:
rxnconsosub[:5]

In [ ]:
check = ['2666961']
rxnconsosub[rxnconsosub.RXAUI.isin(check)]

In [ ]:
rxnrel[rxnrel.RXCUI1.notnull()]

In [ ]:
#check = ['38.0']
#rxnconsosub[rxnconsosub.RXCUI.isin(check)]

In [ ]:
#rxnconsosub = rxnconsosub.astype({'RXCUI': 'float', 'RXAUI': 'float'})

In [ ]:
rxnconsosub.info()

In [ ]:
check = ['788773']
rxnrelsub[rxnrelsub.RXAUI1.isin(check)]

#### Merging based on SCUI column for RXNREL and RXNCONSO 

We divide RXNREL.RRF into two parts.

RXNREL contains data where either the RxNorm AUI or the RxNorm CUI is mentioned for the nodes having a relationship. Since we ultimately need to use both the columns, we divide data into two dataframes, one where the AUI are given and one where the CUI are given. After moulding the columns as required we merge the final output of both these dataframes back into one single dataframe.

In [ ]:
rxnrelcui = rxnrelsub[rxnrelsub.RXCUI1.notnull()]

In [ ]:
rxnrelaui = rxnrelsub[rxnrelsub.RXAUI1.notnull()]

In [ ]:
asso_merge1 = pd.merge(rxnrelaui, rxnconsosub, left_on=['RXAUI1','SAB'] , right_on=['RXAUI','SAB'],  how='left')

In [ ]:
asso_merge1.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
asso_merge1['from_SCUI'] = asso_merge1['SCUI']
asso_merge1['from_namespace'] = asso_merge1['SAB']

In [ ]:
asso_merge1.drop(labels=['RXCUI', 'LAT', 'RXAUI', 'SCUI', 'TTY', 'STR', 'CODE'], axis=1, inplace=True)

In [ ]:
asso_merge2 = pd.merge(asso_merge1, rxnconsosub, left_on=['RXAUI2','SAB'] , right_on=['RXAUI','SAB'],  how='left')

In [ ]:
asso_merge2.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
asso_merge2['to_SCUI'] = asso_merge2['SCUI']
asso_merge2['to_namespace'] = asso_merge2['SAB']

In [ ]:
#asso_merge2.drop(labels=['RXCUI', 'LAT', 'RXAUI', 'SCUI', 'TTY', 'STR', 'CODE'], axis=1, inplace=True)

In [ ]:
asso_merge3 = pd.merge(rxnrelcui, rxnconsosub, left_on=['RXCUI1','SAB'] , right_on=['RXCUI','SAB'],  how='left')

In [ ]:
asso_merge3.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
asso_merge3['from_SCUI'] = asso_merge3['SCUI']
asso_merge3['from_namespace'] = asso_merge3['SAB']

In [ ]:
asso_merge3.drop(labels=['RXCUI', 'LAT', 'RXAUI', 'SCUI', 'TTY', 'STR', 'CODE'], axis=1, inplace=True)

In [ ]:
asso_merge4 = pd.merge(asso_merge3, rxnconsosub, left_on=['RXCUI2','SAB'] , right_on=['RXCUI','SAB'],  how='left')

In [ ]:
asso_merge4.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
asso_merge4['to_SCUI'] = asso_merge4['SCUI']
asso_merge4['to_namespace'] = asso_merge4['SAB']

In [ ]:
asso_mergef = pd.concat([asso_merge2,asso_merge4], ignore_index=True)

In [ ]:
asso_mergef.drop(labels=['RXCUI', 'LAT', 'RXAUI', 'SCUI', 'TTY', 'STR', 'CODE'], axis=1, inplace=True)

In [ ]:
asso_mergef.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
asso_mergef = asso_mergef[asso_mergef.from_SCUI.notnull()]
asso_mergef = asso_mergef[asso_mergef.to_SCUI.notnull()]

In [ ]:
asso_mergef[:5]

In [ ]:
asso_mergef['from_namespace_rectified'] = asso_mergef['from_namespace'].replace('RxNorm','RXNORM')
asso_mergef['from_namespace_rectified'] = asso_mergef['from_namespace'].replace('MeSH','MSH')
asso_mergef['from_namespace_rectified'] = asso_mergef['from_namespace'].replace('SNOMED CT','SNOMEDCT_US')

In [ ]:
asso_mergef['to_namespace_rectified'] = asso_mergef['to_namespace'].replace('RxNorm','RXNORM')
asso_mergef['to_namespace_rectified'] = asso_mergef['to_namespace'].replace('MeSH','MSH')
asso_mergef['to_namespace_rectified'] = asso_mergef['to_namespace'].replace('SNOMED CT','SNOMEDCT_US')

In [ ]:
asso_mergef.to_csv('Association_Merged_RXN_incomplete.csv', index=False, header=1, encoding='utf-8')

### Merging with MRCONSO 

We perform merging of the association file to MRCONSO (UMLS) to match the CUI (UMLS) to the 'from_code' and the 'to_code' (SCUI in UMLS). This mapping is performed because our UMLS model nodes can be accessed with the CUI.
Once we get the CUI mapping we can create the relationships (the column 'name' in association) and add our namespace (MED-RT in this case) and its verison.

In [ ]:
# Load MRCONSO.RRF into a dataframe
mrconso = pd.read_csv('C:/Users/pc/Desktop/C-BIA/Healthcare/UMLS/UMLS_Data/2021AA/META/MRCONSO.RRF', sep='|', header=None, encoding='utf-8')
mrconso[:5]

In [ ]:
mrconso.columns = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF', '']

In [ ]:
mrconsosub = mrconso[['SCUI','CUI','SAB']]
mrconsosub.info()

In [ ]:
asso_mergef_sub = asso_mergef

In [ ]:
asso_mergef2 = pd.merge(asso_mergef_sub, mrconsosub, left_on=['from_SCUI','from_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [ ]:
asso_mergef2.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
asso_mergef2.info()

In [ ]:
asso_mergef2['from_CUI'] = asso_mergef2['CUI']

In [ ]:
asso_mergef2.drop(labels=['CUI','SCUI'], axis=1, inplace=True)

In [ ]:
asso_mergef2 = pd.merge(asso_mergef2, mrconsosub, left_on=['to_SCUI','to_namespace_rectified'] , right_on=['SCUI','SAB'],  how='left')

In [ ]:
asso_mergef2.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
asso_mergef2['to_CUI'] = asso_mergef2['CUI']

In [ ]:
asso_mergef2.drop(labels=['CUI','SCUI'], axis=1, inplace=True)

In [ ]:
asso_mergef2.info()

In [ ]:
asso_mergef2.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
asso_mergef2.info()

In [ ]:
asso_mergef2['version'] = '2021.09.07.21AA'
asso_mergef2['namespace'] = 'RxNorm'
asso_mergef2['name'] = asso_mergef2['REL']
asso_mergef2[:5]

In [ ]:
asso_mergef2.to_csv('Association_Merged_RXN_check.csv', index=False, header=1, encoding='utf-8')

##### Move the "Association_Merged_RXN_check" into the Neo4J import folder

## Importing in Neo4J 

In [ ]:
import pandas as pd

In [ ]:
import getpass
password = getpass.getpass("\nPlease enter the Neo4j database password to continue \n")

In [ ]:
from neo4j import GraphDatabase
driver=GraphDatabase.driver(uri="bolt://localhost:7687", auth=('neo4j',password))
session=driver.session()

In [ ]:
# Create a node for each concept in the UMLS
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///CUIs_preferred_terms.csv" AS COLUMN CREATE (:Concept_UMLS {preferred_term:COLUMN[1], cui:COLUMN[0], semantic_type:COLUMN[2], UMLS_edition:'2021AA'})'''

#session.run(query)

In [ ]:
# Create a uniqueness constraint for the CUI property of each Concept_UMLS node
#query = 'CREATE CONSTRAINT UniqueCUIforConceptConstraint ON (c:Concept_UMLS) ASSERT c.cui IS UNIQUE'
#session.run(query)

In [ ]:
# Create the UMLS string nodes
#query = '''USING PERIODIC COMMIT 100000 LOAD CSV FROM "file:///str_to_CUI.csv" AS COLUMN CREATE (s:String_UMLS {string:COLUMN[0], cui:COLUMN[1], semantic_type:COLUMN[2]})'''

#session.run(query)

###### Sometimes python causes issue and the command below is not executed. In case such a thing happens , it's recommended to run the cell/command again.

To check if these specific relations were added, search in the Neo4J browser for relationship by the name of 'CHD' , 'SIB' , 'SY'. If these 3 exist , most probably all the relationships from the Association_Merged_RXN.csv have been added.

Use the following code to check if the relationship exist: (you can replace CHD with the name of the relationship)

MATCH p=()-[r:CHD]->() RETURN p LIMIT 25

In [ ]:
command = '''USING PERIODIC COMMIT 1000000 LOAD CSV WITH HEADERS FROM "file:///Association_Merged_RXN_check.csv" AS COLUMN MATCH (c1:Concept_UMLS {cui:COLUMN.from_CUI}) MATCH (c2:Concept_UMLS {cui:COLUMN.to_CUI}) OPTIONAL MATCH (c1)-[rel]->(c2) WITH c1, c2, COLUMN, COLLECT(TYPE(rel)) AS relTypes WHERE NOT COLUMN.name IN relTypes CALL apoc.create.relationship(c1, COLUMN.name, {source:COLUMN.namespace, version:COLUMN.version, REL_Additional:COLUMN.RELA}, c2) YIELD rel as newrel RETURN c1, c2, newrel;'''
session.run(command)

### Regarding the Relationships created after executing the code

The following link contains information about what the relationship names stand for: https://www.nlm.nih.gov/research/umls/knowledge_sources/metathesaurus/release/abbreviations.html#mrdoc_REL

